# Sudoku ver. 1

### 원리
빈 칸이 없을 때까지 반복 > 9번 반복 > 빈 칸 개수만큼 반복<br>
{
1) 한 칸 남은 행, 열, 구역 있는지 조사 -> 있으면 채우기 <br>
2) 특정 숫자 해당하는 지역, 칸, 열 다 지움<br>
3) 해당 지역에 가능한 숫자가 그 숫자밖에 없다면 (ex)지역에 자리 하나 남음, 열에 하나 남음, 행에 하나 남음) 채우기<br>
4) 채워지면 2)로 돌아가 해당 숫자 재검증하기, 3)의 경우 발생하면 같은 과정 반복<br>
}
### 결과
- 교수님께서 주신 문제는 풀리지만 인터넷 난이도 상 문제 풀리지 않음

## 1. 함수 정의

### 데이터 처리 관련 함수

In [1]:
import numpy as np

def preProcessing(problem):
    # 문자열 데이터를 받아 9x9의 2차원 np array로 반환
    data = problem.split(',')
    frame = np.reshape(data,(9,9)).astype('int')
    return frame

def splitFrame(frame):
    '''
    9x9의 2차원 int형 array를 받아, dictionary 두 개를 반환
    
    1) num_area
    - key: 1 2 3 / 4 5 6 / 7 8 9 (나눠진 지역의 위치)
    - value: 해당 위치의 3x3 지역의 값들로 이루어진 2차원 array
      [:3,:3]  [:3,3:6]  [:3,6:9]
      [3:6,:3] [3:6,3:6] [3:6,6:9]
      [6:9,:3] [6:9,3:6] [6:9,6:9]
      
    2) num_index
    - key: 1 2 3 / 4 5 6 / 7 8 9 (나눠진 지역의 위치)
    - value: 해당 위치의 3x3 지역의 값들의 인덱스로 이루어진 list in list
      [[r1,r2,r3],[c1,c2,c3]] * 9
    '''
    num_area={}
    num_index={}
    for i in range(3):
        r=(i+1)*3
        for j in range(3):
            c=(j+1)*3
            key=j+i*3+1
            num_area[key]=frame[i*3:r,j*3:c]
            num_index[key]=[list(range(i*3,r)),list(range(j*3,c))]
    return num_area, num_index

def printFrame(frame):
    # 스도쿠 그려줌
    for i in range(9):
        for j in range(9):
            if j==8:
                print(frame[i][j],'| ')
            elif (j+1)%3==0:
                print(frame[i][j],'| ',end='')
            else:
                print(frame[i][j],' ',end='')
        if (i+1)%3==0:
            print('- - - - - - - - - - - - - - -')

### 특정 값을 찾아주는 함수

In [2]:
def findArea(frame,r_index,c_index):
    '''row와 column의 index 값을 넣어 주면 어느 area에 있는지 번호를 찾아 int로 반환함'''
    Num_Area, Num_Index = splitFrame(frame)
    for i in range(1,10):
        area=Num_Index[i]
        r=area[0]
        c=area[1]
        if r_index in r and c_index in c:
            return i
            
def findAreaIndex(frame,a):
    '''
    frame과 area를 넣으면 해당 area의 index 목록을 list로 반환함
    return -> [[r1,c1],[r2,c2],...] , type: list
    '''
    num_area, num_index=splitFrame(frame)
    area=num_index[a]
    r=area[0]
    c=area[1]
    index=[]
    for i in range(3):
        for j in range(3):
            index.append([r[i],c[j]])
    return index

def findEmptyIndex(frame):
    '''
    frame을 넣어주면 값을 채워야 할 행, 열의 인덱스 목록(0인 목록)을 np array로 반환함
    return -> [[r1,c1],[r2,c2],...] , type: numpy.ndarray
    '''
    empty_list=[]
    row, col = np.where(frame==0)
    for i in range(len(row)):
        empty_list.append([row[i],col[i]])
    return np.array(empty_list)

def findNone(array):
    '''
    행, 열 구역에서 없는 숫자를 찾아 숫자들의 리스트를 반환함
    만일 숫자가 하나라면, 정수 하나를 반환함
    '''
    num=list(range(1,10))
    if len(array)==9:
        for i in range(9):
            if array[i] in num:
                num.remove(array[i])
    else:
        for i in range(3):
            for j in range(3):
                if array[i][j] in num:
                    num.remove(array[i][j])
    if len(num)==1:
        return num[0]
    return num

### 세어주는 함수

In [3]:
def countEmpty(r,c,area):
    '''행, 열, 구역의 빈 칸의 개수를 각각 반환함'''
    r_=np.where(r==0) # 튜플이구나,,,
    c_=np.where(c==0)
    r__,c__=np.where(area==0)
    return len(r_[0]), len(c_[0]), len(r__)

def countNum(frame,num):
    """현재 스도쿠 퍼즐에서 특정 숫자(num)의 개수를 반환함"""
    count=0
    for i in range(9):
        for j in range(9):
            if frame[i][j]==num:
                count+=1
    return count

### 제거해주는 함수

In [4]:
def delEmpty(empty_list,l):
    '''
    l에 포함되는 인덱스를 제거한 empty_list를 np array로 반환함
    l은 제거할 row, column index의 쌍으로 이루어진 list in list -> [[r,c],...]
    '''
    new_list=[]
    for x in empty_list:
        new_list.append([x[0],x[1]])
    for x in l:
        new_list.remove(x)
    return np.array(new_list)
    
def delSquares(frame,r,c,area):
    '''
    r, c -> 1*9 np array
    area -> 특정 구역인 3*3 np array
    frame 복사 후, r(행), c(열), area(구역)에 해당하는 부분들의 값을 100으로 바꾼 후 반환
    후에 해당하는 칸을 제외시키고 싶을 때 써먹을 수 있음
    '''
    new_frame=frame.copy()
    for i in range(len(r)):
        new_frame[r[i]]=100
    for i in range(len(c)):
        new_frame[:,c[i]]=100
    for n in area:
        index=findAreaIndex(frame,n)
        for x in index:
            new_frame[x[0]][x[1]]=100
    return new_frame

def appendDelList(r,c,row,column,num_index,del_index):
    '''
    r, c -> 특정 칸의 인덱스, type: int
    row, column, num_index, del_index -> 제외할 항목을 담은 list들, type: list
    '''
    del_index.append([r,c])
    row.append(r)
    column.append(c)
    num_index+=findAreaIndex(frame,findArea(frame,r,c))

## 2. 코드

In [5]:
import numpy as np

print("         ** 9x9 스도쿠 **")
print(" -----------------------------------")
print("| - 스도쿠의 1~9열을 순서대로 입력  |")
print("| - 빈 공간은 0으로 설정            |")
print("| - 구분은 콤마(,):                 |")
print(" -----------------------------------")

while(1):
    data=str(input())
    if len(data)==81*2-1:
        break;
    else:
        print("숫자가 {}개입니다. 81개를 입력하세요".formet(len(data)))

# 데이터 정리
frame=preProcessing(data)
Num_Area, Num_Index=splitFrame(frame) # 3*3 지역

# 스도쿠에 있는 각 숫자의 개수
num_count=[]
for i in range(9):
    num_count.append(countNum(frame,i+1))
    
# 빈 칸의 인덱스
# empty_list[:,0] = r, empty_list[:,1] = c
empty_list=findEmptyIndex(frame) # 변수 바꾸지 말 것 -> delEmpty함수에 그대로 전달

print('')
print('problem: ')
printFrame(frame)
print('')

while(len(empty_list)>0):
    for i in range(9): # 각 숫자에 대해 일단 한 바퀴 조사할 것임
        #print("num: ", i+1)

        # 해당 숫자가 있는 <칸>의 행,열 인덱스 목록 정리 (변.바.말)
        row, column=list(np.where(frame==i+1))
        # 리스트로 바꿔야 함수 안에서 바뀌네요
        row=list(row) 
        column=list(column)

        # 해당 숫자가 있는 <구역>의 인덱스 목록 정리
        area=[] # list -> [3,5,9,8]
        num_index=[] # list in list -> [[r,c]...] (변.바.말)
        for j in range(len(row)):
            area.append(findArea(frame,row[j],column[j]))
            num_index+=findAreaIndex(frame,area[j])

        empty_len=[100,len(empty_list)] # 이걸 왜하냐? while문 끝낼 타이밍을 판별하기 위함
        now=1 # empty_len의 인덱스... 즉 현재 while문의 횟수

        while(empty_len[now]<empty_len[now-1]): # 전 인덱스보다 짧은 경우에만 시행: while문 내에서 빈 칸이 채워지지 않는다면, empty_len이 그대로일 것이므로
            del_index=[] # 채워져서 empty_list에서 제거할 인덱스 리스트, list in list -> [[r,c],...]
            empty_r=empty_list[:,0]
            empty_c=empty_list[:,1]
            for k in range(len(empty_list)):
                r=empty_r[k] # empty index중에서 k번째 r
                c=empty_c[k] # empty index중에서 k번째 c
                n=findArea(frame,r,c) # 현재 있는 <빈칸>이 어느 구역에 있는지. 즉, [r][c]가 몇 번 구역에 있는지
                e_r,e_c,e_a=countEmpty(frame[r],frame[:,c],Num_Area[n]) # 행, 열, 구역에서 빈칸의 개수
                if e_r==1: # 만약 행에서 한 칸 남았다면
                    frame[r][c]=findNone(frame[r]) # 행에 없는 숫자를 빈칸에 넣어줌
                    #print("1-> ({},{})에 {}이 채워졌습니다".format(r+1,c+1,frame[r][c]))
                    appendDelList(r,c,row,column,num_index,del_index)
                elif e_c==1:
                    frame[r][c]=findNone(frame[:,c])
                    #print("2-> ({},{})에 {}이 채워졌습니다".format(r+1,c+1,frame[r][c]))
                    appendDelList(r,c,row,column,num_index,del_index)
                elif e_a==1:
                    frame[r][c]=findNone(Num_Area[n])
                    #print("3-> ({},{})에 {}이 채워졌습니다".format(r+1,c+1,frame[r][c]))
                    appendDelList(r,c,row,column,num_index,del_index)

                # 더 채울 게 없다면, 숫자로 판별
                # row, col, num_index는 모두 숫자 (i+1)이 있는 구역이다. 따라서 해당 구역을 이번 인덱스([r,c])가 피해야 함
                elif (r not in row) and (c not in column) and ([r,c] not in num_index): 
                    # 해당 숫자가 있는 행, 열, 구역들을 지워줌
                    temp_frame=delSquares(frame,row,column,area) # 임시 frame -> 해당 숫자 기피 지역을 모두 100으로 바꾼 상황
                    Temp_Area, Temp_Index=splitFrame(temp_frame)
                    e_r,e_c,e_a=countEmpty(temp_frame[r],temp_frame[:,c],Temp_Area[n])
                    if e_r==1 or e_c==1 or e_a==1:
                        #print('e_r: ',e_r,'e_c: ',e_c,'e_a: ',e_a)
                        frame[r][c]=i+1
                        #print("4-> ({},{})에 {}이 채워졌습니다".format(r+1,c+1,frame[r][c]))
                        appendDelList(r,c,row,column,num_index,del_index)

            empty_list=delEmpty(empty_list,del_index)
            empty_len.append(len(empty_r))
            now+=1

print('answer: ')
printFrame(frame)

         ** 9x9 스도쿠 **
 -----------------------------------
| - 스도쿠의 1~9열을 순서대로 입력  |
| - 빈 공간은 0으로 설정            |
| - 구분은 콤마(,):                 |
 -----------------------------------
0,0,0,0,0,0,0,5,0,7,0,9,6,0,2,0,8,0,3,0,5,0,7,0,6,1,9,8,0,0,2,0,0,3,0,0,0,0,0,1,3,5,0,0,0,0,0,3,0,0,8,0,0,6,5,3,4,0,8,0,2,0,1,0,8,0,3,0,1,5,0,4,0,6,0,0,0,0,0,0,0

problem: 
0  0  0 | 0  0  0 | 0  5  0 | 
7  0  9 | 6  0  2 | 0  8  0 | 
3  0  5 | 0  7  0 | 6  1  9 | 
- - - - - - - - - - - - - - -
8  0  0 | 2  0  0 | 3  0  0 | 
0  0  0 | 1  3  5 | 0  0  0 | 
0  0  3 | 0  0  8 | 0  0  6 | 
- - - - - - - - - - - - - - -
5  3  4 | 0  8  0 | 2  0  1 | 
0  8  0 | 3  0  1 | 5  0  4 | 
0  6  0 | 0  0  0 | 0  0  0 | 
- - - - - - - - - - - - - - -

answer: 
6  4  8 | 9  1  3 | 7  5  2 | 
7  1  9 | 6  5  2 | 4  8  3 | 
3  2  5 | 8  7  4 | 6  1  9 | 
- - - - - - - - - - - - - - -
8  9  1 | 2  6  7 | 3  4  5 | 
4  7  6 | 1  3  5 | 9  2  8 | 
2  5  3 | 4  9  8 | 1  7  6 | 
- - - - - - - - - - - - - - -
5  3  4 | 7  8  